In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.datasets import mnist, cifar10

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import missingno as msno
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC, SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import BaggingClassifier, VotingClassifier

In [5]:
df = pd.read_csv('gaming.csv')

In [6]:
df.columns = map(str.lower, df.columns)

In [7]:
df['gender'] = df['gender'].map({'Male': 1, 'Female':0})

In [8]:
df['gamegenre'].value_counts()

gamegenre
Sports        8048
Action        8039
Strategy      8012
Simulation    7983
RPG           7952
Name: count, dtype: int64

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40034 entries, 0 to 40033
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   playerid                   40034 non-null  int64  
 1   age                        40034 non-null  int64  
 2   gender                     40034 non-null  int64  
 3   location                   40034 non-null  object 
 4   gamegenre                  40034 non-null  object 
 5   playtimehours              40034 non-null  float64
 6   ingamepurchases            40034 non-null  int64  
 7   gamedifficulty             40034 non-null  object 
 8   sessionsperweek            40034 non-null  int64  
 9   avgsessiondurationminutes  40034 non-null  int64  
 10  playerlevel                40034 non-null  int64  
 11  achievementsunlocked       40034 non-null  int64  
 12  engagementlevel            40034 non-null  object 
dtypes: float64(1), int64(8), object(4)
memory usag

In [10]:
X = df.drop(columns=['gamegenre'])
y = df['gamegenre']
y = pd.get_dummies(y, dtype=int)

In [11]:
df.location.unique()

array(['Other', 'USA', 'Europe', 'Asia'], dtype=object)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=.8, random_state=42)

In [13]:
ct = ColumnTransformer([
    ('pl', Pipeline([
        ('si', SimpleImputer()),
        ('sc', StandardScaler()),
        ('pf', PolynomialFeatures(degree=2, include_bias=False))]),['age', 'sessionsperweek', 'playerlevel']),
        ('oh', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'), ['gamedifficulty', 'location', 'engagementlevel'])
])

In [14]:
X_train_ct = ct.fit_transform(X_train)
X_test_ct = ct.transform(X_test)

In [15]:
X_test_ct.shape

(8007, 16)

In [16]:
y_test.shape

(8007, 5)

In [17]:
X_train.shape

(32027, 12)

In [18]:
rf = RandomForestClassifier()

In [19]:
rf.fit(X_train_ct, y_train)

RandomForestClassifier()

In [20]:
rf.score(X_train_ct, y_train)

0.9809535704249539

In [21]:
rf.score(X_test_ct, y_test)

0.024228799800174845

In [22]:
y_pred = rf.predict(X_test_ct)

In [23]:
preds = pd.DataFrame(y_pred, columns=y.columns)

In [24]:
test_df = y_test.idxmax(axis=1)

In [25]:
test_df.value_counts(normalize=True)

Action        0.203822
Strategy      0.202323
Simulation    0.200075
Sports        0.198826
RPG           0.194954
Name: proportion, dtype: float64

In [26]:
new_df = preds.idxmax(axis=1)

In [27]:
new_df.value_counts(normalize=True)

Action        0.899088
Simulation    0.026477
Sports        0.026102
RPG           0.024354
Strategy      0.023979
Name: proportion, dtype: float64

In [28]:
model = Sequential()
model.add(Dense(32, input_shape=(X_train_ct.shape[1],), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
              
history = model.fit(X_train_ct, y_train,
                    validation_data=(X_test_ct, y_test),
                    batch_size=128,
                    epochs=10)

Epoch 1/10


/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


251/251 ━━━━━━━━━━━━━━━━━━━━ 0s 589us/step - accuracy: 0.1939 - loss: 1.7596 - val_accuracy: 0.2013 - val_loss: 1.6104
Epoch 2/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 0s 401us/step - accuracy: 0.2023 - loss: 1.6203 - val_accuracy: 0.2028 - val_loss: 1.6097
Epoch 3/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 0s 392us/step - accuracy: 0.1996 - loss: 1.6132 - val_accuracy: 0.2059 - val_loss: 1.6094
Epoch 4/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 0s 392us/step - accuracy: 0.2009 - loss: 1.6128 - val_accuracy: 0.1987 - val_loss: 1.6096
Epoch 5/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 0s 392us/step - accuracy: 0.2004 - loss: 1.6106 - val_accuracy: 0.2009 - val_loss: 1.6096
Epoch 6/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 0s 390us/step - accuracy: 0.1999 - loss: 1.6105 - val_accuracy: 0.2008 - val_loss: 1.6095
Epoch 7/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 0s 389us/step - accuracy: 0.2071 - loss: 1.6094 - val_accuracy: 0.2036 - val_loss: 1.6094
Epoch 8/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 0s 391us/step - accuracy: 0.2032 - loss: 1.6096 - val_accurac

In [31]:
y_pred_nn = model.predict(X_test_ct)

251/251 ━━━━━━━━━━━━━━━━━━━━ 0s 201us/step


In [39]:
df_model = pd.DataFrame(y_pred_nn, columns=y.columns)

In [40]:
df_model = round(df_model, 2)

In [46]:
df_model.value_counts(normalize=True)

Action  RPG   Simulation  Sports  Strategy
0.20    0.20  0.20        0.20    0.20        0.684776
        0.19  0.20        0.20    0.20        0.024479
              0.21        0.20    0.20        0.022355
        0.20  0.21        0.19    0.20        0.019608
                          0.20    0.20        0.014862
                                                ...   
              0.20        0.21    0.18        0.000125
                          0.19    0.21        0.000125
              0.19        0.22    0.20        0.000125
                          0.21    0.19        0.000125
0.22    0.22  0.18        0.19    0.20        0.000125
Name: proportion, Length: 160, dtype: float64

In [ ]:
if not os.path.exists('models'):
    os.mkdir('models')
    
with open('models/austen-poe-detector.pkl', 'wb') as f:
    pickle.dump(model, f)

In [35]:
pgrid = {
    'base_estimator__max_depth': np.append(np.arange(1, 8), None),
    'base_estimator__min_samples_leaf': np.arange(1, 12, 2)
    
}
""
tree = DecisionTreeClassifier()
bag = BaggingClassifier(base_estimator=tree, n_estimators=100, random_state=2022)
gs = GridSearchCV(bag, pgrid, cv=5, n_jobs=4)

In [36]:
gs.fit(X_train_ct, y_train)

ValueError: 
All the 240 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
96 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_bagging.py", line 337, in fit
    return self._fit(X, y, self.max_samples, sample_weight=sample_weight)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_bagging.py", line 393, in _fit
    y = self._validate_y(y)
        ^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_

In [46]:
print(gs.score(X_train_ct, y_train))
print(gs.score(X_test_ct, y_test))

0.2280766028309742
0.2048156659006894


In [33]:
y_preds = gs.predict(X_test_ct)

NameError: name 'gs' is not defined